In [1]:
import pandas as pd 
import numpy as np
import os 
import time
import glob

In [3]:
#fastas = glob.glob("/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/prg1_ligation/results_ligation/alignment/junc_unmapped/parn-LI*m1000.unmapped.fa")
fastas = glob.glob("/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_SNAP_ligation_07_24_2023/alignment/junc_unmapped/parn-LI_17_uni.xartifacts.xc.genome.aligned.v0.m1.unmapped.aligned.junctions.v0.m1.unmapped.fa")
#fastas = glob.glob("/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_SNAP_ligation_07_24_2023/alignment/junc_unmapped/WT-LI_17_uni.xartifacts.xc.genome.aligned.v0.m1.unmapped.aligned.junctions.v0.m1.unmapped.fa")
#fastas = glob.glob("/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/prg1_ligation/results_ligation/alignment/junc_unmapped/WT-LI*m1000.unmapped.fa")
fastas

['/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/results_SNAP_ligation_07_24_2023/alignment/junc_unmapped/parn-LI_17_uni.xartifacts.xc.genome.aligned.v0.m1.unmapped.aligned.junctions.v0.m1.unmapped.fa']

In [4]:
piRNA_ta_index = '/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/prg1_ligation/tailor_index/uniq_piRNA_50nt'
piRNA_bt_index = '/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/prg1_ligation/bowtie_index/uniq_piRNA_50nt'
genome_bt_index = '/fs/ess/PCON0160/ben/pipelines/nextflow_smRNA/index/bowtie/c_elegans.PRJNA13758.WS279.genomic/c_elegans.PRJNA13758.WS279.genomic'
tmpdir = '/fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/prg1_ligation/tmp'



In [5]:
def process_ligation(fastas, piRNA_ta_index, piRNA_bt_index, genome_bt_index, tmpdir) : 
    
    for e,f in enumerate(fastas) : 
        
        sample = os.path.basename(f).split(".")[0]
        print(sample)
        
        cmd = f"sh /fs/ess/PCON0160/ben/projects/antisense_piRNA_v2/prg1_ligation/tailor_map.sh {f} {piRNA_ta_index} {piRNA_bt_index} {genome_bt_index} {tmpdir}"
        os.system(cmd)
        
        candidates = pd.read_csv(f"{tmpdir}/candidates.bed", 
                                 names = ['chrom', 'start', 'end', 'full_seq', 'count', 'strand', 'seq_minus_unal', 'unal'],
                                 sep = "\t",
                                 header = None
                                )
        
        print(f"Candidates with piRNA sequence {candidates['count'].sum()}")
        
        aln_anti = pd.read_csv(f"{tmpdir}/anti.bed",
                               names = ['chrom', 'anti_start', 'anti_end', 'seq'],
                               sep = "\t",
                               usecols = [0,1,2,3],
                               header = None
                              )
        
        aln_anti[['full_seq','unal','x_count']] = aln_anti.seq.str.split(":" ,expand=True)
        aln_anti = aln_anti.drop(columns = ['seq'])
        
        candidates_filt = aln_anti.merge(candidates, on = ['chrom', 'full_seq', 'unal'], how = 'inner')
        
        #print(candidates.shape[0])
        #print(aln_anti.shape[0])
        #print(candidates_filt.shape[0])
        #print(candidates_filt.head())
        
        candidates_filt['sample'] = sample
        print(f"Candidates with piRNA AND anti sequence {candidates_filt['count'].sum()}")
        if e == 0 : 
            res = candidates_filt 
        else : 
            res = pd.concat([res,candidates_filt], ignore_index = True)
        
    return res

In [6]:
res = process_ligation(fastas, piRNA_ta_index, piRNA_bt_index, genome_bt_index, tmpdir)

parn-LI_17_uni



The following have been reloaded with a version change:
  1) python/3.9-2022.05 => python/3.6-conda5.2



Number of starting reads = 3408789.00 
Number of reads with tails greater than 10nt long = 383.00 


# reads processed: 7365
# reads with at least one alignment: 6346 (86.16%)
# reads that failed to align: 1019 (13.84%)
# reads with alignments suppressed due to -m: 4227 (57.39%)
Reported 2119 alignments


Number of reads uniquely map to genome = 2463.00 
Number of reads mapping antisense to a piRNA = 1871.00 
Candidates with piRNA sequence 8698
Candidates with piRNA AND anti sequence 1858


# reads processed: 2119
# reads with at least one alignment: 1636 (77.21%)
# reads that failed to align: 483 (22.79%)
# reads with alignments suppressed due to -m: 1 (0.05%)
Reported 1635 alignments


In [7]:
res.sort_values(by = ['count'], ascending = False)

,chrom,anti_start,anti_end,full_seq,unal,x_count,start,end,count,strand,seq_minus_unal,sample
20,"WBGene00045637,Y64G10A.19,21ur-3513",9,29,TTAGTTCTTTGCATAGTACCTTCCAAAACGTTTTGGAAGGTACTATGCA,GTTTTGGAAGGTACTATGCA,8,0,29,8,+,TTAGTTCTTTGCATAGTACCTTCCAAAAC,parn-LI_17_uni
736,"WBGene00048838,Y7A9A.17,21ur-1343",9,27,TGAAGGAAGAGTACGAAACTTTTGTAATTACAAAAGTTTCGTACT,TTACAAAAGTTTCGTACT,8,0,27,8,+,TGAAGGAAGAGTACGAAACTTTTGTAA,parn-LI_17_uni
574,"WBGene00048268,Y105C5A.274,21ur-977",10,27,TTCGTGACACATCGCTAGTCCATTAACGTTAATGGACTAGCGAT,GTTAATGGACTAGCGAT,8,0,27,8,+,TTCGTGACACATCGCTAGTCCATTAAC,parn-LI_17_uni
89,"WBGene00045894,Y116A8C.60,21ur-3545",9,28,TCATAGAGTAAGATTTGTTCTGTCTAATATTAGACAGAACAAATCTT,ATTAGACAGAACAAATCTT,7,0,28,7,+,TCATAGAGTAAGATTTGTTCTGTCTAAT,parn-LI_17_uni
737,"WBGene00048838,Y7A9A.17,21ur-1343",9,28,TGAAGGAAGAGTACGAAACTTTTGTAAATTTACAAAAGTTTCGTACT,TTTACAAAAGTTTCGTACT,6,0,28,6,+,TGAAGGAAGAGTACGAAACTTTTGTAAA,parn-LI_17_uni
...,...,...,...,...,...,...,...,...,...,...,...,...
541,"WBGene00048186,Y67A10A.76,21ur-208",10,28,TATGTGTTCCTTAAGCAGATGTCATCTTATGACATCTGCTTAA,CTTATGACATCTGCTTAA,1,0,25,1,+,TATGTGTTCCTTAAGCAGATGTCAT,parn-LI_17_uni
540,"WBGene00048186,Y67A10A.76,21ur-208",10,28,TATGTGTTCCTTAAGCAGATGTCACTTATGACATCTGCTTAA,CTTATGACATCTGCTTAA,1,0,24,1,+,TATGTGTTCCTTAAGCAGATGTCA,parn-LI_17_uni
539,"WBGene00048186,Y67A10A.76,21ur-208",10,27,TATGTGTTCCTTAAGCAGATGTCATTTATGACATCTGCTTAA,TTATGACATCTGCTTAA,1,0,25,1,+,TATGTGTTCCTTAAGCAGATGTCAT,parn-LI_17_uni
538,"WBGene00048186,Y67A10A.76,21ur-208",10,27,TATGTGTTCCTTAAGCAGATGTCATAATTATGACATCTGCTTAA,TTATGACATCTGCTTAA,1,0,27,1,+,TATGTGTTCCTTAAGCAGATGTCATAA,parn-LI_17_uni


In [8]:
res.groupby(['sample']).agg({'chrom':'size'})

,chrom
sample,
parn-LI_17_uni,1533


In [9]:
res.groupby(['sample']).agg({'count':'sum'})

,count
sample,
parn-LI_17_uni,1858


In [10]:
res

,chrom,anti_start,anti_end,full_seq,unal,x_count,start,end,count,strand,seq_minus_unal,sample
0,"WBGene00045523,M199.10,21ur-4900",9,28,TTGAGCATCAATTATCATTCTAATATGAATTAGAATGATAATT,ATGAATTAGAATGATAATT,1,0,24,1,+,TTGAGCATCAATTATCATTCTAAT,parn-LI_17_uni
1,"WBGene00045531,Y105C5A.35,21ur-3262",8,28,TCGAGGATTAATCGAGAATGAATTTTGAAAAATTCATTCTCGATTA,GAAAAATTCATTCTCGATTA,1,0,26,1,+,TCGAGGATTAATCGAGAATGAATTTT,parn-LI_17_uni
2,"WBGene00045532,C02B10.7,21ur-116",9,25,TGGGGTTGTAGATATAGAAGACTTATATAAGTCTTCTATATCT,TAAGTCTTCTATATCT,1,0,27,1,+,TGGGGTTGTAGATATAGAAGACTTATA,parn-LI_17_uni
3,"WBGene00045538,Y51H4A.36,21ur-783",10,26,TAGGACACATCCCAGCCTGTCTTAGTAAGACAGGCTGGGA,GTAAGACAGGCTGGGA,2,1,25,2,+,TAGGACACATCCCAGCCTGTCTTA,parn-LI_17_uni
4,"WBGene00045541,C35D6.15,21ur-2515",13,29,TCTCATATGGTAACTTTATTTCCACGTTGTAAACAACGTGGAAATAAAG,CAACGTGGAAATAAAG,1,0,33,1,+,TCTCATATGGTAACTTTATTTCCACGTTGTAAA,parn-LI_17_uni
...,...,...,...,...,...,...,...,...,...,...,...,...
1528,"WBGene00174958,Y17G9A.84,21ur-9338",9,27,TAGGTACTGAACTTGTCTGAATTTATGAAATTCAGACAAGTT,ATGAAATTCAGACAAGTT,1,0,24,1,+,TAGGTACTGAACTTGTCTGAATTT,parn-LI_17_uni
1529,"WBGene00174964,Y73F8A.1157,21ur-7551",9,27,TGGTTCCTTCCTTCCGCCTTTGGTATATATATACCAAAGGCGGAAGG,TATACCAAAGGCGGAAGG,1,0,29,1,+,TGGTTCCTTCCTTCCGCCTTTGGTATATA,parn-LI_17_uni
1530,"WBGene00174964,Y73F8A.1157,21ur-7551",10,23,TGGTTCCTTCCTTCCGCCTTTGGTATATACCAAAGGCGGAAG,CCAAAGGCGGAAG,1,0,29,1,+,TGGTTCCTTCCTTCCGCCTTTGGTATATA,parn-LI_17_uni
1531,"WBGene00174989,C47A4.140,21ur-8457",8,30,TCTTTCTTTGTAACCCATTAATCATTGTCAACAATGATTAATGGGT...,CAACAATGATTAATGGGTTACA,1,0,28,1,+,TCTTTCTTTGTAACCCATTAATCATTGT,parn-LI_17_uni


In [43]:
res_filt = res.query(
    'seq_minus_unal.str.len() == 21'
).reset_index(drop = True) 

In [44]:
res_filt

,chrom,anti_start,anti_end,full_seq,unal,x_count,start,end,count,strand,seq_minus_unal,sample


In [31]:
res.groupby(['sample']).agg({'count':'sum'})

,count
sample,
parn-LI_17_uni,1858


In [12]:
res_filt.to_csv("prg1_ligation.tsv", sep = "\t", header = True, index = False)

In [13]:
res_filt

,chrom,anti_start,anti_end,full_seq,unal,start,end,count,strand,seq_minus_unal,sample


In [14]:
res_filt.groupby(['chrom', 'sample']).agg( count = ('count', 'sum')).reset_index().to_csv("parn1_ligation_counts.tsv", sep = "\t", header = True, index = False)